In [ ]:
%pip install wikipedia-api
%pip install torch

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=d5a7dd6d527192fcf9a77f60fcac16ac0479a9d532be8b758bad7ec74df5fd50
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [ ]:
import wikipediaapi
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re

# Initialize the Qwen2-7B-Instruct model with FP16 to reduce memory usage
model_name = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Move the model to CUDA (GPU) or keep on CPU based on availability
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Model is using: {device}")

# Update tokenizer properties
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

def generate(messages, **gen_kwargs):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt", padding=True).to(device)
    gen_kwargs['temperature'] = gen_kwargs.get("temperature", 0.01)
    gen_kwargs['max_new_tokens'] = gen_kwargs.get("max_new_tokens", 256)

    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            attention_mask=model_inputs.attention_mask,
            **gen_kwargs
        )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"Generated Response:\n{response}\n")
    return response

def fetch_wikipedia_page(title):
    wiki_wiki = wikipediaapi.Wikipedia(language='en', user_agent='docvis/1.0 (your_email@example.com)')
    page = wiki_wiki.page(title)
    return page.text if page.exists() else None

def clean_and_validate_json(response):
    try:
        cleaned_response = response.strip().replace("```json", "").replace("```", "").strip()
        while True:
            try:
                if not cleaned_response.endswith('}'):
                    cleaned_response += '}'
                if cleaned_response.count('{') != cleaned_response.count('}'):
                    if cleaned_response.count('{') > cleaned_response.count('}'):
                        cleaned_response += '}'
                    else:
                        cleaned_response = '{' + cleaned_response.lstrip('{')
                if cleaned_response.count('"') % 2 != 0:
                    cleaned_response += '"'
                data = json.loads(cleaned_response)
                return data
            except json.JSONDecodeError:
                print("Incomplete or invalid JSON detected, attempting to fix or remove the last entry...")
                if ',' in cleaned_response:
                    cleaned_response = re.sub(r',[^,]*}]?}?$', '}]}', cleaned_response)
                else:
                    break
        return {"Relationships": [{"Person1": "Unknown", "Person2": "Unknown", "Relationship": "No Data"}]}
    except Exception as e:
        print(f"Unexpected Error: {e}")
        print(f"Failed to parse the response. Original Response:\n{response}")
        return {"Relationships": []}

def extract_relationships_with_transformer(text):
    system_prompt = """
    You are a helpful assistant that extracts and logs relationships between entities (such as people, places, events, and their associations) from the given text.
    Please return relationships in the following format:
    ```json
    {
      "Relationships": [
        {
          "Person1": "<Entity 1>",
          "Person2": "<Entity 2>",
          "Relationship": "<relation type>"
        }
      ]
    }
    ```
    Example:
    {
      "Relationships": [
        {
          "Person1": "Neil Armstrong",
          "Person2": "Buzz Aldrin",
          "Relationship": "First two men to land on the moon together"
        },
        {
          "Person1": "Apollo 11",
          "Person2": "United States",
          "Relationship": "Spaceflight conducted by"
        }
      ]
    }
    Ensure that the keys are exactly "Relationships", "Person1", "Person2", and "Relationship".
    Extract at least 8 relationships if possible.
    """
    user_prompt = f"Extract relationships from the following text:\n\n{text[:1500]}"

    response = generate([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ])

    relationships = clean_and_validate_json(response)
    print(f"Extracted Relationships:\n{json.dumps(relationships, indent=2)}\n")
    return relationships

if __name__ == "__main__":
    title = "Apollo 11"
    text = fetch_wikipedia_page(title)
    if text:
        result = extract_relationships_with_transformer(text)
        print(f"Final Output:\n{json.dumps(result, indent=2)}")
    else:
        print("Failed to retrieve the Wikipedia page.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model is using: cuda
Generated Response:
{
  "Relationships": [
    {
      "Person1": "Apollo 11",
      "Person2": "United States",
      "Relationship": "Conducted by"
    },
    {
      "Person1": "Neil Armstrong",
      "Person2": "Buzz Aldrin",
      "Relationship": "Landed on the Moon together"
    },
    {
      "Person1": "Neil Armstrong",
      "Person2": "Apollo 11",
      "Relationship": "First man to land on the Moon"
    },
    {
      "Person1": "Buzz Aldrin",
      "Person2": "Apollo 11",
      "Relationship": "Second man to land on the Moon"
    },
    {
      "Person1": "Apollo 11",
      "Person2": "Michael Collins",
      "Relationship": "Crewed mission with"
    },
    {
      "Person1": "Apollo 11",
      "Person2": "Kennedy Space Center",
      "Relationship": "Launched from"
    },
    {
      "Person1": "Apollo 11",
      "Person2": "Saturn V rocket",
      "Relationship": "Launched by"
    },
   

Incomplete or invalid JSON detected, attempting to fix or remov

In [ ]:
import json
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files

# Function to load relationships from JSON data
def load_relationships(json_data):
    return json_data["Relationships"]

# Function to create a NetworkX graph from relationships
def create_graph(relationships):
    G = nx.Graph()
    for rel in relationships:
        G.add_edge(rel["Person1"], rel["Person2"], relationship=rel.get("Relationship", "Unknown"))
    return G

# Function to create and display a radial graph using Plotly
def create_radial_graph(G):
    pos = nx.spring_layout(G)
    edge_x = []
    edge_y = []
    edge_text = []  # To hold hover text for edges
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"Relationship: {edge[2]['relationship']}<br>{edge[0]} ↔ {edge[1]}")  # Add relationship to hover text

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        hovertext=edge_text,  # Set hover text for edges
        mode='lines'
    )

    node_x = []
    node_y = []
    node_text = []
    for node, (x, y) in pos.items():
        node_x.append(x)
        node_y.append(y)

        # Collect information about connections for each node
        connections = []
        for neighbor in G.neighbors(node):
            relationship = G.edges[node, neighbor].get('relationship', 'Unknown')
            connections.append(f"Connected to {neighbor} for relationship - {relationship}")

        # Join all connection info and add to hover text
        connections_info = "<br>".join(connections)
        node_text.append(f"Name: {node}<br>{connections_info}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,  # Set hover text for nodes with connection information
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2)
    )

    # Set node color based on the number of connections
    node_adjacencies = [len(adj[1]) for adj in G.adjacency()]
    node_trace.marker.color = node_adjacencies

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<b>Apollo 11 Relationships - Radial Graph</b>',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            text="Radial Graph - Entity Relationships",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# Function to create and display a scatter plot using Plotly
def create_scatter_plot(relationships):
    # Generate unique entity nodes and assign an index
    entities = set()
    for rel in relationships:
        entities.add(rel["Person1"])
        entities.add(rel["Person2"])

    entity_to_index = {entity: i for i, entity in enumerate(entities)}

    # Initialize traces for edges and nodes
    edge_x = []
    edge_y = []
    edge_text = []  # To hold hover text for edges

    # Create edges between nodes
    for rel in relationships:
        x0, y0 = entity_to_index[rel["Person1"]], 0
        x1, y1 = entity_to_index[rel["Person2"]], 1
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"Relationship: {rel.get('Relationship', 'Unknown')}<br>{rel['Person1']} ↔ {rel['Person2']}")

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        hovertext=edge_text,  # Set hover text for edges
        mode='lines'
    )

    node_x = list(entity_to_index.values())
    node_y = [0] * len(entity_to_index)
    node_text = []  # To hold hover text for nodes

    # Collect information about connections for each node
    for node in entity_to_index:
        connections = []
        for rel in relationships:
            if rel["Person1"] == node:
                connections.append(f"Connected to {rel['Person2']} for relationship - {rel.get('Relationship', 'Unknown')}")
            elif rel["Person2"] == node:
                connections.append(f"Connected to {rel['Person1']} for relationship - {rel.get('Relationship', 'Unknown')}")

        # Join all connection info and add to hover text
        connections_info = "<br>".join(connections)
        node_text.append(f"Name: {node}<br>{connections_info}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,  # Set hover text for nodes with connection information
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2)
    )

    # Set node color based on degree (number of connections)
    node_adjacencies = [len([rel for rel in relationships if rel["Person1"] == node or rel["Person2"] == node])
                        for node in entity_to_index.keys()]
    node_trace.marker.color = node_adjacencies

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<b>Apollo 11 Relationships - Scatter Plot</b>',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# Function to generate and save both graphs
def generate_and_display_graphs(relationships):
    G = create_graph(relationships)

    # Create individual figures
    radial_fig = create_radial_graph(G)
    scatter_fig = create_scatter_plot(relationships)

    # Display both figures simultaneously
    radial_fig.show()
    scatter_fig.show()

    # Save both graphs as HTML files for download
    radial_fig.write_html("radial_relationship_graph.html")
    scatter_fig.write_html("scatter_relationship_graph.html")
    print("Graphs saved as 'radial_relationship_graph.html' and 'scatter_relationship_graph.html'")
    files.download("radial_relationship_graph.html")
    files.download("scatter_relationship_graph.html")

# Example usage
json_data = {
    "Relationships": [
        {"Person1": "Apollo 11", "Person2": "United States", "Relationship": "Conducted by"},
        {"Person1": "Neil Armstrong", "Person2": "Buzz Aldrin", "Relationship": "Landed on the Moon together"},
        {"Person1": "Neil Armstrong", "Person2": "Apollo 11", "Relationship": "First man to land on the Moon"},
        {"Person1": "Buzz Aldrin", "Person2": "Apollo 11", "Relationship": "Second man to land on the Moon"},
        {"Person1": "Apollo 11", "Person2": "Michael Collins", "Relationship": "Crewed mission with"},
        {"Person1": "Apollo 11", "Person2": "Kennedy Space Center", "Relationship": "Launched from"},
        {"Person1": "Apollo 11", "Person2": "Saturn V rocket", "Relationship": "Launched by"}
    ]
}

# Load relationships from JSON data and display graphs
relationships = load_relationships(json_data)
generate_and_display_graphs(relationships)

Graphs saved as 'radial_relationship_graph.html' and 'scatter_relationship_graph.html'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files

# Function to load relationships from JSON data
def load_relationships(json_data):
    return json_data["Relationships"]

# Function to create a NetworkX graph from relationships
def create_graph(relationships):
    G = nx.Graph()
    for rel in relationships:
        G.add_edge(rel["Person1"], rel["Person2"], relationship=rel.get("Relationship", "Unknown"))
    return G

# Function to create and display a radial graph using Plotly
def create_radial_graph(G):
    pos = nx.spring_layout(G)
    edge_x = []
    edge_y = []
    edge_text = []  # To hold hover text for edges
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"Relationship: {edge[2]['relationship']}<br>{edge[0]} ↔ {edge[1]}")  # Add relationship to hover text

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        hovertext=edge_text,  # Set hover text for edges
        mode='lines'
    )

    node_x = []
    node_y = []
    node_text = []
    for node, (x, y) in pos.items():
        node_x.append(x)
        node_y.append(y)

        # Collect information about connections for each node
        connections = []
        for neighbor in G.neighbors(node):
            relationship = G.edges[node, neighbor].get('relationship', 'Unknown')
            connections.append(f"Connected to {neighbor} for relationship - {relationship}")

        # Join all connection info and add to hover text
        connections_info = "<br>".join(connections)
        node_text.append(f"Name: {node}<br>{connections_info}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,  # Set hover text for nodes with connection information
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2)
    )

    # Set node color based on the number of connections
    node_adjacencies = [len(adj[1]) for adj in G.adjacency()]
    node_trace.marker.color = node_adjacencies

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<b>Apollo 11 Relationships - Radial Graph</b>',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            text="Radial Graph - Entity Relationships",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# Function to create and display a scatter plot using Plotly
def create_scatter_plot(relationships):
    # Generate unique entity nodes and assign an index
    entities = set()
    for rel in relationships:
        entities.add(rel["Person1"])
        entities.add(rel["Person2"])

    entity_to_index = {entity: i for i, entity in enumerate(entities)}

    # Initialize traces for edges and nodes
    edge_x = []
    edge_y = []
    edge_text = []  # To hold hover text for edges

    # Create edges between nodes
    for rel in relationships:
        x0, y0 = entity_to_index[rel["Person1"]], 0
        x1, y1 = entity_to_index[rel["Person2"]], 1
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"Relationship: {rel.get('Relationship', 'Unknown')}<br>{rel['Person1']} ↔ {rel['Person2']}")

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        hovertext=edge_text,  # Set hover text for edges
        mode='lines'
    )

    node_x = list(entity_to_index.values())
    node_y = [0] * len(entity_to_index)
    node_text = []  # To hold hover text for nodes

    # Collect information about connections for each node
    for node in entity_to_index:
        connections = []
        for rel in relationships:
            if rel["Person1"] == node:
                connections.append(f"Connected to {rel['Person2']} for relationship - {rel.get('Relationship', 'Unknown')}")
            elif rel["Person2"] == node:
                connections.append(f"Connected to {rel['Person1']} for relationship - {rel.get('Relationship', 'Unknown')}")

        # Join all connection info and add to hover text
        connections_info = "<br>".join(connections)
        node_text.append(f"Name: {node}<br>{connections_info}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,  # Set hover text for nodes with connection information
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2)
    )

    # Set node color based on degree (number of connections)
    node_adjacencies = [len([rel for rel in relationships if rel["Person1"] == node or rel["Person2"] == node])
                        for node in entity_to_index.keys()]
    node_trace.marker.color = node_adjacencies

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<b>Apollo 11 Relationships - Scatter Plot</b>',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# Function to generate and save both graphs
def generate_and_display_graphs(relationships):
    G = create_graph(relationships)

    # Create individual figures
    radial_fig = create_radial_graph(G)
    scatter_fig = create_scatter_plot(relationships)

    # Display both figures simultaneously
    radial_fig.show()
    scatter_fig.show()

    # Save both graphs as HTML files for download
    radial_fig.write_html("radial_relationship_graph.html")
    scatter_fig.write_html("scatter_relationship_graph.html")
    print("Graphs saved as 'radial_relationship_graph.html' and 'scatter_relationship_graph.html'")
    files.download("radial_relationship_graph.html")
    files.download("scatter_relationship_graph.html")

# Example usage
json_data = {
    "Relationships": [
        {"Person1": "Apollo 11", "Person2": "United States", "Relationship": "Conducted by"},
        {"Person1": "Neil Armstrong", "Person2": "Buzz Aldrin", "Relationship": "Landed on the Moon together"},
        {"Person1": "Neil Armstrong", "Person2": "Apollo 11", "Relationship": "First man to land on the Moon"},
        {"Person1": "Buzz Aldrin", "Person2": "Apollo 11", "Relationship": "Second man to land on the Moon"},
        {"Person1": "Apollo 11", "Person2": "Michael Collins", "Relationship": "Crewed mission with"},
        {"Person1": "Apollo 11", "Person2": "Kennedy Space Center", "Relationship": "Launched from"},
        {"Person1": "Apollo 11", "Person2": "Saturn V rocket", "Relationship": "Launched by"}
    ]
}

# Load relationships from JSON data and display graphs
relationships = load_relationships(json_data)
generate_and_display_graphs(relationships)

Graphs saved as 'radial_relationship_graph.html' and 'scatter_relationship_graph.html'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import json
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files

# Function to load relationships from JSON data
def load_relationships(json_data):
    return json_data["Relationships"]

# Function to create a NetworkX graph from relationships
def create_graph(relationships):
    G = nx.Graph()
    for rel in relationships:
        G.add_edge(rel["Person1"], rel["Person2"], relationship=rel.get("Relationship", "Unknown"))
    return G

# Function to create and display a radial graph using Plotly
def create_radial_graph(G):
    pos = nx.spring_layout(G)
    edge_x = []
    edge_y = []
    edge_text = []  # To hold hover text for edges
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"Relationship: {edge[2]['relationship']}<br>{edge[0]} ↔ {edge[1]}")  # Add relationship to hover text

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        hovertext=edge_text,  # Set hover text for edges
        mode='lines'
    )

    node_x = []
    node_y = []
    node_text = []
    for node, (x, y) in pos.items():
        node_x.append(x)
        node_y.append(y)

        # Collect information about connections for each node
        connections = []
        for neighbor in G.neighbors(node):
            relationship = G.edges[node, neighbor].get('relationship', 'Unknown')
            connections.append(f"Connected to {neighbor} for relationship - {relationship}")

        # Join all connection info and add to hover text
        connections_info = "<br>".join(connections)
        node_text.append(f"Name: {node}<br>{connections_info}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,  # Set hover text for nodes with connection information
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2)
    )

    # Set node color based on the number of connections
    node_adjacencies = [len(adj[1]) for adj in G.adjacency()]
    node_trace.marker.color = node_adjacencies

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<b>Apollo 11 Relationships - Radial Graph</b>',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            text="Radial Graph - Entity Relationships",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# Function to create and display a scatter plot using Plotly
def create_scatter_plot(relationships):
    # Generate unique entity nodes and assign an index
    entities = set()
    for rel in relationships:
        entities.add(rel["Person1"])
        entities.add(rel["Person2"])

    entity_to_index = {entity: i for i, entity in enumerate(entities)}

    # Initialize traces for edges and nodes
    edge_x = []
    edge_y = []
    edge_text = []  # To hold hover text for edges

    # Create edges between nodes
    for rel in relationships:
        x0, y0 = entity_to_index[rel["Person1"]], 0
        x1, y1 = entity_to_index[rel["Person2"]], 1
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"Relationship: {rel.get('Relationship', 'Unknown')}<br>{rel['Person1']} ↔ {rel['Person2']}")

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='text',
        hovertext=edge_text,  # Set hover text for edges
        mode='lines'
    )

    node_x = list(entity_to_index.values())
    node_y = [0] * len(entity_to_index)
    node_text = []  # To hold hover text for nodes

    # Collect information about connections for each node
    for node in entity_to_index:
        connections = []
        for rel in relationships:
            if rel["Person1"] == node:
                connections.append(f"Connected to {rel['Person2']} for relationship - {rel.get('Relationship', 'Unknown')}")
            elif rel["Person2"] == node:
                connections.append(f"Connected to {rel['Person1']} for relationship - {rel.get('Relationship', 'Unknown')}")

        # Join all connection info and add to hover text
        connections_info = "<br>".join(connections)
        node_text.append(f"Name: {node}<br>{connections_info}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,  # Set hover text for nodes with connection information
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2)
    )

    # Set node color based on degree (number of connections)
    node_adjacencies = [len([rel for rel in relationships if rel["Person1"] == node or rel["Person2"] == node])
                        for node in entity_to_index.keys()]
    node_trace.marker.color = node_adjacencies

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<b>Apollo 11 Relationships - Scatter Plot</b>',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# Function to generate and save both graphs
def generate_and_display_graphs(relationships):
    G = create_graph(relationships)

    # Create individual figures
    radial_fig = create_radial_graph(G)
    scatter_fig = create_scatter_plot(relationships)

    # Display both figures simultaneously
    radial_fig.show()
    scatter_fig.show()


# Load relationships from JSON data and display graphs
relationships = load_relationships(json_data)
generate_and_display_graphs(relationships)